In [15]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Möbius Plateau Auto-Explorer Ωᴾ² (75-dp, 3-level towers, 70 ops)\n",
    "# Paste or edit raw_rows below then Runtime ▶ Run-all.  No widgets.\n",
    "\n",
    "import mpmath as mp, math, numpy as np, pandas as pd, itertools, json, os\n",
    "mp.mp.dps = 75  # crank precision\n",
    "\n",
    "# –– 1) RAW SCAN TABLE (bits, k, φ_micro, λ, rel_err) ––\n",
    "raw_rows = [\n",
    "    (64, 3.6, 0.003942, 0.0, 0.4117525875906678269),\n",
    "    (64, 3.8, 0.003942, 0.0, 0.4117525875906678269),\n",
    "    (64, 4.0, 0.003942, 0.0, 0.4117525875906678269),\n",
    "    (64, 4.2, 0.003942, 0.0, 0.4117525875906678269),\n",
    "    (128,3.6, 0.003942, 0.0, 0.4117525875906678271),\n",
    "    (128,3.8, 0.003942, 0.0, 0.4117525875906678271),\n",
    "    (128,4.0, 0.003942, 0.0, 0.4117525875906678271),\n",
    "    (128,4.2, 0.003942, 0.0, 0.4117525875906678271),\n",
    "]\n",
    "df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])\n",
    "display(df.head())\n",
    "\n",
    "# –– 2) CONSTANTS & helpers ––\n",
    "phi   = (1+mp.sqrt(5))/2;  phi_inv = 1/phi\n",
    "alpha = 1/137.035999084  # CODATA fine-structure 1/α\n",
    "alpha_inv = 1/alpha\n",
    "delta = 4.669201609      # Feigenbaum δ\n",
    "alpha_F = 2.502907875    # Feigenbaum α\n",
    "pi  = mp.pi;     e = mp.e;     gamma  = 0.577215664901532\n",
    "FIB_BIN = sum(int(c)*2**-i for i,c in enumerate('010010100101'[:40],1))  # 40-bit rabbit\n",
    "def comp(x): return 1-x   # complement 1-x\n",
    "\n",
    "# –– 3) BUILD ARITH / SCALE / POWER-TOWER OPS ––\n",
    "OPS = {}\n",
    "def add(name, fn): OPS[name]=fn\n",
    "\n",
    "# scalers × / ÷ 3,9,10\n",
    "for n in (3,9,10):   add(f\"×{n}\", lambda x,n=n: x*n)\n",
    "for n in (3,9,10):   add(f\"÷{n}\", lambda x,n=n: x/n)\n",
    "\n",
    "# roots / powers\n",
    "add('√x',  lambda x: mp.sqrt(x));   add('∛x', lambda x: mp.nroot(x,3))\n",
    "add('x¹ᐟ⁹', lambda x: x**(1/9));    add('x⁻¹ᐟ⁹', lambda x: x**(-1/9))\n",
    "\n",
    "# complement & inverse\n",
    "add('c', comp);       add('x⁻¹', lambda x: 1/x)\n",
    "\n",
    "# (x-100)/10 trick\n",
    "add('-100÷10', lambda x: (x-100)/10)\n",
    "\n",
    "# 3-level power-towers (short)\n",
    "def pwr2(x):  return x**x if x>0 and x<10 else x\n",
    "def pwr3(x):  xx=pwr2(x); return xx**x if xx>0 and xx<10 else xx\n",
    "add('xˣ', pwr2);  add('xˣˣ', pwr3)\n",
    "\n",
    "# attach constants (numeric + complement + inverse)\n",
    "consts = {\n",
    "    'φ':phi, 'φ-¹':phi_inv, 'α':alpha, 'α⁻¹':alpha_inv,\n",
    "    'δ':delta, 'α_F':alpha_F, 'π':pi, 'e':e, 'γ':gamma, 'β':FIB_BIN,\n",
    "}\n",
    "for name,val in consts.items():\n",
    "    add(name, lambda x,val=val: val)                # standalone\n",
    "    add(f'×{name}', lambda x,val=val: x*val)        # scale\n",
    "    add(f'÷{name}', lambda x,val=val: x/val)        # div\n",
    "    add(f'{name}·c', lambda x,val=val: comp(val))   # comp(const)\n",
    "\n",
    "print(f\"Loaded {len(OPS)} ops.\")\n",
    "\n",
    "# –– 4) TAG PLATEAUS & RUN ––\n",
    "def tag_plateaus(frame: pd.DataFrame, eps:float):\n",
    "    base = frame['rel_err'].min(); out = frame.copy()\n",
    "    out['plateau'] = ((out['rel_err']-base)/eps).astype(int)\n",
    "    return out\n",
    "\n",
    "views=[]\n",
    "for eps in (1e-4,5e-6,1e-7,5e-9):\n",
    "    tagged = tag_plateaus(df, eps)\n",
    "    for op,fn in OPS.items():\n",
    "        tmp = tagged.copy(); tmp['ε']=eps; tmp['op']=op\n",
    "        tmp['rel_err'] = tmp['rel_err'].apply(fn)  # element-wise op\n",
    "        views.append(tmp.iloc[:80])                # keep first 80 rows per view for eyeball\n",
    "\n",
    "result = pd.concat(views, ignore_index=True)\n",
    "\n",
    "# save to Excel\n",
    "raw_file = 'plateaus_raw_v14.xlsx'; ops_file='plateaus_ops_v14.xlsx'\n",
    "df.to_excel(raw_file,index=False); result.to_excel(ops_file,index=False)\n",
    "print(f\"Files written: {raw_file} and {ops_file}\")\n",
    "result.head(30)"
   ]
  }
 ],
 "metadata": {
  "colab": {},
  "kernelspec": {
   "display_name": "python3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined